In [ ]:
import sys, os
sys.path.extend([os.path.abspath('../')])

In [ ]:
import numpy as np
import sympy as sy
import sparse as sp
import math

In [ ]:
from qgs.params.params import QgParams
from qgs.functions.tendencies import create_tendencies

In [ ]:
model_parameters = QgParams(dynamic_T=True)

In [ ]:
model_parameters.set_atmospheric_channel_fourier_modes(2, 2, mode="symbolic")
# Mode truncation at the wavenumber 2 in the x and at the 
# wavenumber 4 in the y spatial coordinates for the ocean
model_parameters.set_oceanic_basin_fourier_modes(2, 4, mode="symbolic")

In [ ]:
model_parameters.symbolic_params

In [ ]:
from qgs.inner_products.symbolic import AtmosphericSymbolicInnerProducts, OceanicSymbolicInnerProducts
from qgs.tensors.qgtensor import QgsTensor, QgsTensorDynamicT, QgsTensorT4
from qgs.tensors.symbolic_qgtensor import SymbolicTensorLinear, SymbolicTensorDynamicT
from qgs.tensors.symbolic_qgtensor import _shift_dict_keys, _add_to_dict

In [ ]:
%%time
# Takes ~5 mins
# atm_ip = AtmosphericSymbolicInnerProducts(model_parameters)
# ocn_ip_stored = OceanicSymbolicInnerProducts(model_parameters, stored=True, return_symbolic=True)
ocn_ip = OceanicSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
%%time
# Takes ~5 mins
# atm_ip_stored = AtmosphericSymbolicInnerProducts(model_parameters, return_symbolic=True)
atm_ip = AtmosphericSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
# atm_ip_stored.save_to_file('temp_atm_sym_de.ip')
# ocn_ip_stored.save_to_file('temp_ocn_sym_de.ip')

In [ ]:
atm_loaded = AtmosphericSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)
ocn_loaded = OceanicSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
atm_loaded.load_from_file('temp_atm_sym_de.ip')
ocn_loaded.load_from_file('temp_ocn_sym_de.ip')

## Testing the symbolic_qgtensor

In [ ]:
ten = SymbolicTensorDynamicT(params=model_parameters, atmospheric_inner_products=atm_loaded, oceanic_inner_products=ocn_loaded)

In [ ]:
_, _, ten_num = create_tendencies(model_parameters, return_qgtensor=True)

In [ ]:
# ten.test_tensor_numerically(tol=1e-9)

In [ ]:
# atm_num = AtmosphericSymbolicInnerProducts(model_parameters, stored=True)
# ocn_num = OceanicSymbolicInnerProducts(model_parameters, stored=True)

# ocn_num_ns = OceanicSymbolicInnerProducts(model_parameters, stored=False)

In [ ]:
model_parameters.ground_basis is None

In [ ]:
from qgs.functions.symbolic_output import create_symbolic_equations, translate_equations

In [ ]:
model_parameters.dynamic_T

In [ ]:
%%time
f = create_symbolic_equations(model_parameters, atm_loaded, ocn_loaded)

In [ ]:
python_lang_translation = {
    'sqrt': 'math.sqrt',
    'pi': 'math.pi'
}

In [ ]:
def translate_equations_my(equations, params, language='python'):

    _base_file_path = ''
    if language == 'python':
        file_ext = '.py'
        base_file = _base_file_path + file_ext

        # translate mathematical operations
        str_eq = dict()
        for i in range(1, params.ndim):
            print(i)
            temp_str = str(equations[i])
            for k in python_lang_translation.keys():
                temp_str = temp_str.replace(k, python_lang_translation[k])
            str_eq[i] = temp_str
    return str_eq

In [ ]:
f[0]

In [ ]:
translate_equations_my(f[0], model_parameters)

In [ ]:
len(f)